In [1]:
import pandas as pd
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import numpy as np

import matplotlib.ticker as mtick
from adjustText import adjust_text

# Set display options
pd.options.display.float_format = '{:,.2f}'.format

# Load your data
clark_county_ts2021 = pd.read_csv("H:/My Drive/7. SNV Industry Study/SNV_ArcGIS/IHS Files/Clark_County_TS2021.csv")
bridges_excel = pd.ExcelFile("H:/My Drive/7. SNV Industry Study/SNV_ArcGIS/IHS Files/bridges.xlsx")
US_2020_CSA = pd.read_csv("H:/My Drive/7. SNV Industry Study/SNV_ArcGIS/IHS Files/2020_US_CSA.csv")

# Load necessary sheets from the Excel file
stcc = bridges_excel.parse("STCC")
regions = bridges_excel.parse("Regions")
modes = bridges_excel.parse("Modes")
county_region = bridges_excel.parse("County to Region")

# Checking for duplicates
clark_county_ts2021 = clark_county_ts2021.drop_duplicates()
US_2020_CSA = US_2020_CSA.drop_duplicates()

# Define origin_regions and destination_regions separately
origin_regions = regions.rename(columns={"Region": "Origin Region", "Region Name": "Origin Region Name"})
destination_regions = regions.rename(columns={"Region": "Destination Region", "Region Name": "Destination Region Name"})

# Merge origin_regions and destination_regions into clark_county_ts2021
clark_county_ts2021 = pd.merge(clark_county_ts2021, origin_regions, on='Origin Region', how='left')
clark_county_ts2021 = pd.merge(clark_county_ts2021, destination_regions, on='Destination Region', how='left')
# Merge the summary by mode with the modes table
clark_county_ts2021 = pd.merge(clark_county_ts2021, modes, how='left', on='Mode')
# Rename the rest of the columns
clark_county_ts2021.rename(columns={
    'State_x': 'Origin State', 
    'BEA_x': 'Origin BEA', 
    'BEA Name_x': 'Origin BEA Name', 
    'Country_x': 'Origin Country', 
    'State_y': 'Destination State', 
    'BEA_y': 'Destination BEA', 
    'BEA Name_y': 'Destination BEA Name', 
    'Country_y': 'Destination Country'}, inplace=True)

# Print DataFrame to check
#print(clark_county_ts2021)


# Outbound: Origin is Clark County
clark_county_outbound = clark_county_ts2021[
    (clark_county_ts2021['Origin Region Name'] == "Clark County, NV") & 
    (clark_county_ts2021['Destination Region Name'] != "Clark County, NV")]

# Inbound: Destination is Clark County
clark_county_inbound = clark_county_ts2021[
    (clark_county_ts2021['Destination Region Name'] == "Clark County, NV") & 
    (clark_county_ts2021['Origin Region Name'] != "Clark County, NV")]

# Through: Clark County is neither origin nor destination
clark_county_through = clark_county_ts2021[
    (clark_county_ts2021['Origin Region Name'] != "Clark County, NV") & 
    (clark_county_ts2021['Destination Region Name'] != "Clark County, NV")]

# Intra: Both origin and destination is Clark County
clark_county_intra = clark_county_ts2021[
    (clark_county_ts2021['Origin Region Name'] == "Clark County, NV") & 
    (clark_county_ts2021['Destination Region Name'] == "Clark County, NV")]

In [3]:
# Group by Year and Mode Name for each filter and calculate the sum of Values and Tons
outbound_summary = clark_county_outbound.groupby(['Year', 'Name'])[['Value', 'Tons']].sum().reset_index()
inbound_summary = clark_county_inbound.groupby(['Year', 'Name'])[['Value', 'Tons']].sum().reset_index()
through_summary = clark_county_through.groupby(['Year', 'Name'])[['Value', 'Tons']].sum().reset_index()
intra_summary = clark_county_intra.groupby(['Year', 'Name'])[['Value', 'Tons']].sum().reset_index()

# Rename the Value and Tons columns for each filter
outbound_summary.rename(columns={'Value': 'OB_Value', 'Tons': 'OB_Tons'}, inplace=True)
inbound_summary.rename(columns={'Value': 'IB_Value', 'Tons': 'IB_Tons'}, inplace=True)
through_summary.rename(columns={'Value': 'Thr_Value', 'Tons': 'Thr_Tons'}, inplace=True)
intra_summary.rename(columns={'Value': 'Intr_Value', 'Tons': 'Intr_Tons'}, inplace=True)

# Merge all the DataFrames on Year and Name
CC_Modes_Summary = pd.merge(outbound_summary, inbound_summary, on=['Year', 'Name'], how='outer')
CC_Modes_Summary = pd.merge(CC_Modes_Summary, through_summary, on=['Year', 'Name'], how='outer')
CC_Modes_Summary = pd.merge(CC_Modes_Summary, intra_summary, on=['Year', 'Name'], how='outer')

# Print the final DataFrame
print(CC_Modes_Summary)


    Year             Name         OB_Value      OB_Tons          IB_Value  \
0   2015              Air   193,002,111.88    43,442.48    432,173,476.98   
1   2015            Other     1,105,507.75       161.74      1,130,102.55   
2   2015         Rail NEC    29,826,546.57    19,090.05    142,296,950.98   
3   2015      Truck L-T-L   389,739,232.94   198,735.28  1,799,805,204.49   
4   2015        Truck NEC   810,242,035.85   148,251.40    702,908,346.18   
5   2015        Truck PVT 1,327,261,612.20 3,189,621.48  9,605,560,442.99   
6   2015  Truck Truckload 3,451,744,427.90 2,584,410.50 15,541,554,740.93   
7   2019              Air   223,403,981.99    45,247.92    413,675,785.11   
8   2019            Other     1,016,583.05       144.53      1,149,906.46   
9   2019         Rail NEC    16,606,664.37    10,531.54    142,784,976.70   
10  2019      Truck L-T-L   456,206,769.49   242,834.88  1,815,425,888.92   
11  2019        Truck NEC   609,825,796.98   116,353.58    971,264,579.85   

In [4]:
# Rearrange the columns
CC_Modes_Summary = CC_Modes_Summary[['Year', 'Name', 'IB_Value', 'OB_Value', 'Intr_Value', 'Thr_Value', 'IB_Tons', 'OB_Tons', 'Intr_Tons', 'Thr_Tons']]

# Print the final DataFrame
CC_Modes_Summary


,Year,Name,IB_Value,OB_Value,Intr_Value,Thr_Value,IB_Tons,OB_Tons,Intr_Tons,Thr_Tons
0,2015,Air,"432,173,476.98","193,002,111.88",nan,nan,"62,091.72","43,442.48",nan,nan
1,2015,Other,"1,130,102.55","1,105,507.75",nan,"10,220.39",315.54,161.74,nan,23.60
2,2015,Rail NEC,"142,296,950.98","29,826,546.57",nan,"10,682,702,524.03","221,483.69","19,090.05",nan,"1,785,968.63"
3,2015,Truck L-T-L,"1,799,805,204.49","389,739,232.94","203,976,818.90","9,431,805,186.25","670,940.84","198,735.28","117,465.25","2,494,063.06"
4,2015,Truck NEC,"702,908,346.18","810,242,035.85",nan,"12,546,356,729.52","154,999.31","148,251.40",nan,"2,849,743.18"
5,2015,Truck PVT,"9,605,560,442.99","1,327,261,612.20","3,501,665,070.12","25,569,067,565.51","7,352,933.91","3,189,621.48","11,348,456.71","15,734,787.43"
6,2015,Truck Truckload,"15,541,554,740.93","3,451,744,427.90","4,459,673,667.71","63,478,941,434.41","9,164,008.92","2,584,410.50","4,637,783.28","27,838,555.31"
7,2019,Air,"413,675,785.11","223,403,981.99",nan,nan,"58,964.76","45,247.92",nan,nan
8,2019,Other,"1,149,906.46","1,016,583.05",nan,"5,528.76",325.98,144.53,nan,12.77
9,2019,Rail NEC,"142,784,976.70","16,606,664.37",nan,"9,036,557,318.15","209,544.18","10,531.54",nan,"1,543,632.89"
